# **Connect to GDrive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Torrent downloader by libtorrent**

In [ ]:
!apt install python3-libtorrent
!pip install libtorrent

In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)

download_path={1:"/content/",2:'/content/drive/MyDrive/'}
#1. donwload_path[1]:saves locally in Colab
#2. download_path[2]:saves files in drive root folder
params = {
    'save_path': download_path[1],
    'storage_mode': lt.storage_mode_t(2),
    }

#input for magnet link
link="magnet:?xt=urn:btih:B81FB629E5767C65602D39874B63E2C6943E312D&dn=%5BAnimeRG%5D+Boku+no+Hero+Academia+Season+-+2+%281-25%29+%28My+Hero+Academia+14-38%29+%5B720p%5D+%5BEng+Dubbed%5D+%5BJRR%5D&tr=http%3A%2F%2Fnyaa.tracker.wf%3A7777%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2740%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fcoppersurfer.tk%3A6969%2Fannounce"

# if link =="":link=input()
# torrent file praser

print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting\n", handle.name())

current_file=str(handle.name())
while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('\r%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]),end="")
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

# **MAKING A SINGLE ZIP**

In [ ]:
import pathlib
from zipfile import ZipFile
import os

#current_file="data"
source=f"/content/{current_file}"#"/content/sample_data"#
dir=download_path[2]#"/content/"
file_directory=pathlib.Path(source)
print(file_directory)

remove_option=1
#trevarse folder for files and directory
with ZipFile(f"{dir}{current_file}.zip", mode="w") as ZipAchive:
    for file_path in file_directory.rglob("*"):
        if ".ipynb_checkpoints" in str(file_path): continue
        print(file_path)
        ZipAchive.write(file_path,arcname=file_path.relative_to(file_directory))
        if os.path.isfile(file_path) and remove_option:
          os.remove(file_path)

#print dir
with ZipFile(f"{dir}{current_file}.zip", mode="r") as archive:
    archive.printdir()

# **SPLITING BINARY FILES**


In [ ]:
current_file="Devil May Cry 5 [FitGirl Repack]"
#make headfile.read() as chunks

In [ ]:
num=0
file_to_zip=f"/content/{current_file}.zip"
dir_to_save=f"/content/drive/MyDrive/PART-{num}"
gb=5 # 1-gigabyte use split part under 5gb due to ram limitations for 8gb use below 2gb
split_size=int(gb*1024**3)
with open (file_to_zip,'rb') as headfile:
    filecount=1
    while True:
        data=headfile.read(split_size)
        print(len(data))
        if not data:
            print(f"{file_to_zip} finished!")
            break
        print(f"{file_to_zip}.{filecount:03d}")
        if (filecount*gb >= 15):
          while True:
            x=input("change file enter safe:")
            if x == "safe":
              num+=1
              break

        with open(f"{dir_to_save}/{current_file}.{filecount:03d}",'wb') as zip_part:
            zip_part.write(data)
            #print(f"{file_to_zip}{filecount}")
        filecount+=1

# **Upload to Pixel Drain**

In [ ]:
import requests
#Pixeldrainurl ="https://pixeldrain.com/api/file"
data='/content/sample_data/mnist_train_small.csv'#f"/content/{current_file}.zip"#input("enter the files path")
files={'file':open(f"{data}",'rb')}
PostFile=requests.post(url="https://pixeldrain.com/api/file",files=files)
print(PostFile.text)
import json
print(f"https://pixeldrain.com/u/{json.loads(PostFile.text)['id']}")

# **ZIP EXTRACTION**

In [ ]:
from zipfile import ZipFile
import os
# Create a ZipFile Object and load sample.zip in it
with ZipFile(f'/content/{currentfile}.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   os.chdir(f'/content/{currentfile}/')
   zipObj.extractall()

# **7 ZIP BASED SPLITING**

In [ ]:
!sudo apt install p7zip-full p7zip-rar
!pip install logzero

In [ ]:
#telegram bot. The file size limit is 50MB in upload by telegram bot The file size limit is 1500MB in upload by telegram client but you may add some text or other info so 1495 is more safe

#! /usr/bin/python3
# -*- coding:utf-8 -*-
# apt-get install p7zip-full

import subprocess
import os
import math
import logzero
import shutil

logger = logzero.logger

MAX_SPLIT_SIZE = 1495

def file_split_7z(file_path, split_size=MAX_SPLIT_SIZE):
    file_path_7z_list = []
    # if origin file is 7z file rename it
    origin_file_path = ""
    if os.path.splitext(file_path)[1] == ".7z":
        origin_file_path = file_path
        file_path = os.path.splitext(origin_file_path)[0] + ".7zo"
        os.rename(origin_file_path, file_path)
    # do 7z compress
    fz = os.path.getsize(file_path) / 1024 / 1024
    pa = math.ceil(fz / split_size)
    head, ext = os.path.splitext(os.path.abspath(file_path))
    archive_head = "".join((head, ext.replace(".", "_"))) + ".7z"
    for i in range(pa):
        check_file_name = "{}.{:03d}".format(archive_head, i + 1)
        if os.path.isfile(check_file_name):
            logger.debug("remove exists file | {}".format(check_file_name))
            os.remove(check_file_name)
    cmd_7z = ["7z", "a", "-v{}m".format(split_size), "-y", "-mx0", archive_head, file_path]
    proc = subprocess.Popen(cmd_7z, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = proc.communicate()
    if b"Everything is Ok" not in out:
        logger.error("7z output | {}".format(out.decode("utf-8")))
        logger.error("7z error | {}".format(err.decode("utf-8")))
        return file_path_7z_list

    for i in range(pa):
        file_path_7z_list.append("{}.{:03d}".format(archive_head, i + 1))
    # if origin file is 7z file rename it back
    if origin_file_path:
        os.rename(file_path, origin_file_path)
    return file_path_7z_list

def do_file_split(file_path, split_size=MAX_SPLIT_SIZE):
    """caculate split size
        example max split size is 1495 file size is 2000
        than the split part num should be int(2000 / 1495 + 0.5) = 2
        so the split size should be 1000 + 1000 but not 1495 + 505
        with the file size increase the upload risk would be increase too
    """
    file_size = os.path.getsize(file_path) / 2 ** 20
    split_part = math.ceil(file_size / split_size)
    new_split_size = math.ceil(file_size / split_part)
    logger.info("file size | {} | split num | {} | split size | {}".format(file_size, split_part, new_split_size))
    return
    file_path_7z_list = file_split_7z(file_path, split_size=new_split_size)
    return file_path_7z_list
print(do_file_split(f"/content/{current_file}.zip"))